<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
# SETUP
from datascience import *
from prob140 import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pylab
from scipy import stats
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

In [ ]:
def search(x_limits, cdf, u):
    """
    Runs a binary search to find the inverse cdf.
    """
    # Handle possible asymptotes.
    if cdf(x_limits[0]) > u:
        return x_limits[0]
    if cdf(x_limits[1]) < u:
        return x_limits[1]
    
    mid = (x_limits[0] + x_limits[1])/2
    diff = u - cdf(mid)
    if np.abs(diff) < 0.01:
        return mid
    if diff < 0:
        return search((x_limits[0], mid), cdf, u)
    return search((mid, x_limits[1]), cdf, u)

def plot_axes(cdf_table):
    values = cdf_table.column(cdf_table.num_columns - 1)
    cum = list(np.cumsum(values))
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    plt.yticks([0] + cum)
    plt.ylim(-0.1, 1.1)
    plt.plot([0,0], [0,1], color="k", lw=3)
    plt.xlim(-0.02, 1)
    plt.scatter([0]*(len(cum) + 1),
                [0] + cum, s=55, color="k")

def plot_discrete_cdf(cdf_table, u=None):
    """
    Plots the cdf of a discrete distribution.
    
    Parameters
    ----------
    cdf_table : Table
        Table of cdf values.
    u : float
        Value from (0, 1) to plot inverse cdf of.
    """
    values = cdf_table.column(0)
    values = np.append(values[0] - 2, values)

    cum = cdf_table.column(cdf_table.num_columns - 1)
    cum = np.append(0, np.cumsum(cum))

    for i in range(len(values) - 1):
        plt.plot([values[i], values[i+1]], [cum[i], cum[i]],
                 color="darkblue")
        plt.plot([values[i+1], values[i+1]], [cum[i], cum[i+1]],
                 ls="--", color="darkblue" )
    plt.scatter(values, cum, s=50, color="darkblue")    

    plt.plot([values[-1], values[-1] + 2], [1,1],
             color="darkblue")

    plt.xlim(values[0], values[-1] + 2)
    plt.ylim(-0.1, 1.1)
    plt.xlabel('$x$')
    plt.ylabel('CDF at $x$')
    plt.title('Graph of CDF');
    
    if u != None:
        for i in range(len(values)):
            if u <= cum[i]:
                index = values[i]
                break
        height = u
        
        plt.plot([values[0], (index+values[0])/2], [height, height],
                 marker='>', color='red', lw=1)
        plt.plot([(index+values[0])/2, index], [height, height],
                 color='red', lw=1)
        plt.plot([index, index], [height, height/2], marker="v",
                 color="red", lw=1)
        plt.plot([index, index], [0, height/2], color="red", lw=1)

def plot_continuous_cdf(x_limits, cdf, u=None):
    """
    Plots the cdf of a continuous distribution.
    """
    x = np.linspace(*x_limits, 100)
    cdf_values = list(map(cdf, x))
    plt.plot(x, cdf_values, color="darkblue")
    plt.xlabel('$x$')
    plt.ylabel('CDF at $x$')
    plt.title('Graph of CDF');
    
    if not u is None:
        index = search(x_limits, cdf, u)
        height = u

        plt.plot([x_limits[0], (index+x_limits[0])/2],
                 [height, height], marker='>', color='red', lw=1)
        plt.plot([(index+x_limits[0])/2, index],
                 [height, height], color='red', lw=1)
        plt.plot([index, index], [height, height/2],
                 marker="v", color="red", lw=1)
        plt.plot([index, index], [0, height/2], color="red", lw=1)


    plt.xlim(*x_limits)

def unit_interval_to_discrete(cdf_table):
    uniform_slider = widgets.FloatSlider(
        value=0.5,min=0,max=1,step=0.02, description='u')
    @interact(u = uniform_slider)
    def plot(u):
        plot_discrete_cdf(cdf_table, u)


def unit_interval_to_continuous(x_limits, cdf):
    uniform_slider2 = widgets.FloatSlider(
        value=0.5, min=0,max=1,step=0.02, description='u')
    
    @interact(u = uniform_slider2)
    def plot(u):
        if (cdf(u) > x_limits[1] or cdf(u) < x_limits[0]):
            plot_continuous_cdf(x_limits, cdf)
        else:
            plot_continuous_cdf(x_limits, cdf, u)
    
def override_hist(*args, **kwargs):
    """
    This cleans up some unfortunate floating point precision
    bugs in the datascience library and fixes the edgecolor
    bug from matplotlib.
    """
    kwargs['edgecolor'] = 'w'
    Table.hist2(*args, **kwargs)
    ax = plt.gca()
    ticks = ax.get_xticks()
    if np.any(np.array(ticks) != np.rint(ticks)):
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))

if not hasattr(Table, 'hist2'):
    Table.hist2 = Table.hist
    
Table.hist = override_hist

# Lab 9: Simulation and the CDF#

Simulation helps us understand properties of random variables. For example, earlier in the term you saw `simulate_path` for simulating Markov Chains; this was helpful for understanding transition behavior and reversibility. Simulation is also important because properties observed in simulations can lead to the development of new results. 

In this lab you will simulate random variables with specified distributions.

What you will learn:
- How to use `SciPy` for simulation
- How to construct and read graphs of the cumulative distribution functions (cdfs)
- How being able to simulate just one distribution allows us to simulate all others

#newpage

## Part 1: Simulation in SciPy ##
The `stats` module of `SciPy` is familiar to you by now. For any of the well known distributions, you can use `stats` to simulate values of a random variable with that distribution. The general method is `stats.distribution_name.rvs(size = n)` where `rvs` stands for "random variates" and `n` is the number of independent replications you want. 

Every statistical system has conventions for how to specify the parameters of the distribution. In this lab we will tell you the specifications for a few distributions. Later you will be able to see a general pattern in the specifications.

### 1a) Uniform $(0, 1)$ ###
The call is straightforward: `stats.uniform.rvs(0, 1, size=n)`. Complete the cell below to draw the histogram of 100,000 simulated values of a random variable that has the uniform (0, 1) distribution. The `hist` option `bins=25` results in 25 equal bins.

In [ ]:

sim_uniform = ...
sim_uniform_tbl = Table().with_column(
    'Simulated Uniform (0, 1)', sim_uniform
)
sim_uniform_tbl.hist(bins=25)

### 1b) Reading the Scales of the Histogram ###
The unit on the horizontal axis is any unit of length; you can think of it as centimeters if you want, but we will just refer to it as the "unit". Fill in the blanks below and provide units where appropriate. Some units have been provided for you.

(i) The width of each bin is $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$ units.

(ii) The height of each bar is approximately $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$ per $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$.

(iii) Histograms represent percents with $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$, so the answers to (i) and (ii) imply that the percent of simulated values in each bin is approximately $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$%.

(iv) Let the random variable $U$ have the uniform (0, 1) distribution, and let $B$ be any bin of the histogram. The answer to (i) implies that $P(U \in B) = \underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$%.

(v) If instead of `bins=25` we had used `bins=20` as the option to `hist`, then the answer to (iv) would have been $\underline{~~~~~~~~~~~~~~~~~~~~~~~~~~~~}$%


**Your answer here**

### 1c) Normal ###
To simulate normally distributed random variables, use `stats.norm.rvs(mean, sd, size=n)`. Complete the cells below to simulate 100,000 values of a standard normal variable and then 100,000 values of a normal variable that has mean 10 and SD 5. Use the labels `Simulated Standard Normal` and `Simulated Normal (mu 10, sigma 5)` for the appropriate columns. You are welcome to use two different tables in the two cells. 

In [ ]:
...
Table().with_column(...)
...hist(bins=25)
plt.xticks(np.arange(-4, 4.1))
plt.title('Histogram 1');

In [ ]:
...
...hist(bins=25)
plt.title('Histogram 2');

Compare the numbers on the horizontal axes of the two histograms, and fill in the blank.

The value $\underline{~~~~~~~~~~~~~~~~~~~~~}$ on the horizontal axis of Histogram 1 is the same as the value 5 on the horizontal axis of Histogram 2 expressed in $\underline{~~~~~~~~~~~~~~~~~~~~~}$ units.

### 1d) Exponential ###
The two common parametrizations of the exponential are the rate $\lambda$ as we do in Prob 140 or the mean $1/\lambda$. The mean is called the `scale` parameter in `stats`. Run the cell below to simulate 100,000 values of a random variable that has the exponential distribution with rate 0.5

In [ ]:
sim_expon = stats.expon.rvs(scale=1/0.5, size=100000)
sim_expon_tbl = Table().with_column(
    'Simulated Exponential (rate 0.5)', sim_expon
)
sim_expon_tbl.hist(bins=25)
plt.xticks(np.arange(0, 21, 2));

Find the average of the simulated values and check that it is consistent with the rate.

In [ ]:

...

#newpage

### Part 2. The Idea ###
How are all these random numbers generated? In the rest of the lab we will develop a general method that underlies the simulations above, by considering examples of increasing complexity. 

Our starting point is a distribution on just four values. 

Suppose $X$ has the distribution `dist_X` below. 

In [ ]:
vals_X = make_array(-2, 1, 4, 7)
probs_X = make_array(0.3, 0.1, 0.2,0.4)

dist_X = Table().values(vals_X).probability(probs_X)
dist_X

Our goal is to simulate one value of $X$. That is, we want to construct a process that returns one of the four possible values of $X$ with the right probabilities.

### 2a) A Vertical Unit Interval ###
The graphic below shows the probabilities in `dist_X` stacked vertically as in Lab 4. From the bottom to the top, therefore, you have the unit interval.

Now imagine throwing a dart at the unit interval. That is, let $U$ be a random variable that has the uniform distribution on (0, 1), and suppose you mark the value of $U$ on the unit interval shown in the graph.

In [ ]:
plot_axes(dist_X)

Find the following probabilities and see how they are related to the distribution of $X$.

(i) $P(U \le 0.3)$

(ii) $P(0.3 < U \le 0.4)$

(iii) $P(0.4 < U \le 0.6)$

(iv) $P(0.6 < U \le 1)$



i. **Your Answer Here**

ii. **Your Answer Here**

iii. **Your Answer Here**

iv. **Your Answer Here**

### 2b) Idea for Simulating $X$ ###
Starting with a uniform $(0, 1)$ random variable $U$, propose a method of generating a value of $X$. 

Your method should take $U$ as its input and return one of the four possible values as output, in such a way that for each $i = -2, 1, 4, 7$, the chance of returning the value $i$ is $P(X = i)$.

Just describe your method in words. No formula or code is needed.


**Your Answer Here**


Throw a dart uniformly at the unit interval. If it lands in the lowest interval, return -2, if in the next interval then return 1, if in the next interval then return 4, and if in the highest interval then return 7.

#newpage

### Part 3. Visualizing the Idea ###
The method `plot_discrete_cdf` takes a distribution as its argument and plots the cdf.

Run the cell below to get a graph of the cdf of the random variable $X$ in Part 1.

In [ ]:
plot_discrete_cdf(dist_X)

### 3a) Reading the Graph ###
Let $F_X$ be the cdf of $X$. What is the value of $F_X(2.57)$, and is that the value the graph shows?


**Your Answer Here**

### 3b) Jumps ###
At what points $x$ does the graph have a jump? For each point $x$ at which there is a jump, find the size of the jump in terms of the distribution of $X$.


**Your Answer Here**

### 3c) From the Unit Interval to Values of $X$ ###
The function `unit_interval_to_discrete` takes a distribution as its argument and displays an animation of a method that takes a number on the unit interval and returns one of the values of a random variable that has the given distribution. 

Run the cell below. Move the slider around and see how the returned value changes **depending on the starting value** in the unit interval. How is the method that is being displayed related to the one you proposed in Part 1?

In [ ]:
unit_interval_to_discrete(dist_X)

### 3d) A Random Starting Point ###
The method `plot_discrete_cdf` that you used earlier also takes a second argument which is a number between 0 and 1. 

Complete the cell below so that the second argument is picked uniformly at random from (0, 1). Run the cell a few times. How is it related to the method you proposed in Part 1 for generating a value of $X$?

In [ ]:

plot_discrete_cdf(dist_X, ...)


**Your Answer Here**

#newpage

### Part 4. Extension to Continuous Distributions ###
Now suppose you want to generate a random variable that has a specified continuous distribution. Suppose you are given a uniform $(0, 1)$ random number to start with. 

### 4a) Exponential CDF ###
Let $T$ be a random variable that has the exponential distribution with rate $\lambda = 0.5$. Let $F_T$ be the cdf of $T$. Define a function `expon_mean2_cdf` that takes a numerical argument $x$ and returns $F_T(x)$. Use `np.exp(y)` for $e^y$.

Make sure your function works for **all** real numbered values of $x$.

In [ ]:

lamb = 0.5  # don't use "lambda" as that means something else in Python

def expon_mean2_cdf(x):
    ...

The function `plot_continuous_cdf` plots the cdf of a continuous variable. The first two arguments:
- an interval (a, b) over which to draw the cdf
- a cdf function that takes a numerical input and returns the value of the cdf at that input

Run the cell below to check that your function `expon_mean2_cdf` looks good.

In [ ]:
plot_continuous_cdf((-1, 8), expon_mean2_cdf)

### 4b) Idea for Simulating an Exponential Random Variable ###

Suppose you are given one uniform (0, 1) random number and are asked to simulate $T$. Based on Part 3 of the lab, propose a method for doing this.

You don't have to prove that the method works. We'll get to that in lecture. Just propose the method.


**Your Answer Here**

### 4c) Visualizing the Idea ###
The animation in the cell below is analogous to the one in Part 3. It takes a plotting interval and a continuous cdf function as its arguments, and demonstrates a method for picking a number on the positive real line starting with a value on the unit interval that forms the vertical axis. 

Run the cell and move the slider around to see how the returned value changes depending on the starting value on the vertical axis.

In [ ]:
unit_interval_to_continuous((-1, 8), expon_mean2_cdf)

The method `plot_continuous_cdf` takes a third argument which is a number between 0 and 1. 

Complete the cell below so that the third argument is a random number picked uniformly from (0, 1). Run the cell a few times. How is the output related to the method you proposed in (b) for generating a value of $T$?

In [ ]:

...

### 4d) Simulating a Standard Normal Random Variable ###

Complete the cell below so that the random value pointed to by the downwards arrow has the standard normal distribution. 

In [ ]:

def norm_cdf(k):
    """Optional helper function."""
    return ...

plot_continuous_cdf((-3, 3), ...)

### 4e) The General Method ###
Let $F$ be any continuous increasing cdf (that is, no jumps and no flat bits). Let $U$ be a uniform (0, 1) random variable. Fill in the blank:

To construct a random variable $X = g(U)$ so that $X$ has the cdf $F$, take $g = \underline{~~~~~~~~~~~~~}$.

If $F$ is a discrete cdf then the function $g$ is complicated to write out formally, so we're not asking you to do that. The practical description of the method of simulation is in Parts 1 and 2.

#newpage

### Part 5. Empirical Verification that the Method Works ###

### a) The Starting Values ###
Create a table that is called `sim` for simulation and consists of one column called `Uniform` that contains the values of 100,000 i.i.d. uniform (0, 1) random variables.

In [ ]:
N = 100000
u = ...
sim = Table().with_column("Uniform", u)
sim

### b) Tranformation to Exponential ###
Use 4(c) and the values in the column `Uniform` to create an array of values that have the exponential distribution with rate 0.5. This is what is going on "under the hood" in `stats.expon.rvs`.

**Do not** simulate new random numbers, as you will lose the connection with the values in `Uniform`. Use `np.log(y)` for $\log(y)$.

Augment `sim` with a column containing the new array.

In [ ]:

def uniform_to_exponential_mean2(u):
    return ...

exponential_mean2 = sim.apply(...)
sim = sim.with_column('Sim. Exponential (rate 0.5)', exponential_mean2)
sim

Run the cell below and compare with the histogram in 1(d) to confirm that your calculation is correct.

In [ ]:
sim.hist('Sim. Exponential (rate 0.5)', bins=25)
plt.xticks(np.arange(0, 21, 2));

### 5c) Transformation to Standard Normal ###

Use 4(c) and the values in the column `Uniform` to create an array of values that have the standard normal distribution. Augment `sim` with a column containing the new array.

In [ ]:

def norm_ppf(k):
    """Optional helper function."""
    return ...

standard_normal = sim.apply(..., 'Uniform')
sim = sim.with_column('Sim. Standard Normal', standard_normal)
sim

Run the cell below to check that your calculation is correct.

In [ ]:
sim.hist('Sim. Standard Normal', bins=25)

### 5d) Another Normal ###
Now augment `sim` with a column whose contents are the values in `Sim. Standard Normal` transformed so that they have the normal distribution with $\mu = 10$ and $\sigma = 5$. You don't need `apply` for this one.

In [ ]:

z = sim.column('Sim. Standard Normal')
sim = sim.with_column('Sim. Normal (Mu=10, Sigma=5)', ...)
sim

Run the cell below to confirm that your calculations are correct.

In [ ]:
sim.hist('Sim. Normal (Mu=10, Sigma=5)', bins=25)

Finally, go back and look through Part 5. Notice that the only time you generated random numbers was when you simulated 100,000 uniform (0, 1) values. All the other variables were deterministic transformations of the uniform variable.

## Conclusion ##
You have learned that:
- To simulate a random variable with a desired distribution, what you need is a uniform random number generator and the cdf of the desired distribution. You can then use the method of this lab to simulate the value.
- The only random numbers a statistical system needs are uniform on (0, 1). Random numbers from all other distributions follow by the method you have developed in this lab.
- Discrete cdfs consist of jumps and flat parts, at places that you have identified.
- If you have a standard normal variable you can easily transform it to become any other specified normal.

Since uniform $(0, 1)$ random numbers are central to all simulations, their quality is very important for the accuracy and reliability of simulations. Testing and assessing uniform random number generators is serious business, because random number generators don't really produce random numbers. They follow deterministic processes that produce results that have properties that resemble those of random numbers. That is why they are called Pseudo Random Number Generators or PRNGs. [Python uses the Mersenne Twister](https://docs.python.org/3.6/library/random.html), one of the most tested and reliable PRNGs. SciPy uses the [Mersenne Twister for RandomState](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.RandomState.html#numpy.random.RandomState) and draws from a large number of discrete and continuous distributions. Take a look at the list on the RandomState page and see how many you can recognize.

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_09.ipynb")